# **Lung Perfusion Analysis Pipeline**
This notebook is designed for the processing and quantitative analysis of planar lung images, focusing on pulmonary perfusion studies. It employs a combination of image processing techniques and statistical analysis to segment lung regions, divide them into sectors (upper, middle, and lower), and compute key metrics such as area, mean intensity, and cumulative intensity (Kct). The methodology, based on binarization, morphological operations, and connected component analysis, aims to assess the distribution of perfusion and support clinical applications. This approach can aid in diagnosing and monitoring conditions like pulmonary embolism or COPD, with future possibilities of extending the methodology to three-dimensional imaging and advanced segmentation algorithms.

## Setup of the environment

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Segmentation through binarization
`load_and_binarize_image()` handles the segmentation by binarization. The image is loaded in grayscale and binarized to separate the lung regions.

In [1]:
def load_and_binarize_image(filepath):
    """
    Load the image, crop unwanted borders, and apply binarization.
    """
    image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Image not found or could not be loaded.")

    _, binary_image = cv2.threshold(image, 66, 255, cv2.THRESH_BINARY)

    # Display results
    plt.imshow(binary_image, cmap="gray")
    plt.title("Binary Image")
    plt.axis("off")
    plt.show()

    return binary_image

## Application of Morphological Operators
`apply_morphological_operations()` applies morphological operations to the binary image, using a closing operation to reduce noise and enhance lung segmentation.

In [2]:
def apply_morphological_operations(binary_image):
    """
    Use morphological operations to clean the binary image.
    """
    kernel_open1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 11))
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 5))

    # Step 1: Opening to remove noise
    opened_image = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel_open1)

    # Step 2: Closing to fill holes
    processed_image = cv2.morphologyEx(opened_image, cv2.MORPH_CLOSE, kernel_close)

    # Display results
    plt.imshow(processed_image, cmap="gray")
    plt.title("Morphological Operations: Open and Close")
    plt.axis("off")
    plt.show()

    return processed_image

 ## Extraction of Connected Components
 `extract_connected_lung_components()` extracts connected components in the processed binary image. This function assumes that the two largest components correspond to the lungs.

In [3]:
def extract_connected_lung_components(processed_image):
    """
    Extract the connected components, assuming the largest two are the lungs.
    """

    # Connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(processed_image, connectivity=8)

    # Sort components by area
    sorted_indices = np.argsort(stats[:, cv2.CC_STAT_AREA])[::-1][1:3]  # Skip background

    # Lungs extraction as masks
    lung_masks = [(labels == idx).astype(np.uint8) * 255 for idx in sorted_indices]

    # Create a colorized label map for visualization
    mako_colors = sns.color_palette("mako", num_labels)
    mako_colors_normalized = [(int(c[0] * 255), int(c[1] * 255), int(c[2] * 255)) for c in mako_colors]
    label_colored = np.zeros((*processed_image.shape, 3), dtype=np.uint8)
    for label_id in range(1, num_labels):  # Skip background (label 0)
        label_mask = (labels == label_id)
        color = mako_colors_normalized[label_id]
        label_colored[label_mask] = color
    plt.imshow(label_colored)
    plt.title("Connected Components")
    plt.axis("off")
    plt.show()

    return lung_masks

## Definition of Sectors and Creation of Masks
 `define_adaptive_lung_sectors()` divides each lung mask into three sectors (upper, middle, and lower) and creates separate masks for each sector.

In [4]:
def define_adaptive_lung_sectors(lung_mask):
    """
    Divide the lung mask into upper, middle, and lower sectors based on the bounding box of the lung region.
    Visualize the sectors on the lung mask.
    """
    # Find the bounding box of the lung region
    y_coords, x_coords = np.where(lung_mask == 255)
    if y_coords.size == 0 or x_coords.size == 0:
        return [np.zeros_like(lung_mask) for _ in range(3)]  # Return empty sectors if no lung is found

    # Define the bounding box
    y_min, y_max = y_coords.min(), y_coords.max()

    # Define sector height within the bounding box
    lung_height = y_max - y_min + 1
    sector_height = lung_height // 3

    # Create masks for each sector within the bounding box
    sectors = []
    for i in range(3):
        mask = np.zeros_like(lung_mask)
        start_y = y_min + i * sector_height
        end_y = y_min + (i + 1) * sector_height if i < 2 else y_max + 1
        mask[start_y:end_y, :] = lung_mask[start_y:end_y, :]
        sectors.append(mask)

    """# Visualization of sectors
    combined_sectors = np.zeros_like(lung_mask, dtype=np.uint8)
    colors = [50, 150, 255]  # Grayscale values for visualization
    for i, sector in enumerate(sectors):
        combined_sectors[sector > 0] = colors[i]
    plt.imshow(combined_sectors, cmap="gray")
    plt.title("Lung Sectors (Upper, Middle, Lower)")
    plt.axis("off")
    plt.show()"""

    return sectors

## Extraction of Characteristics
`compute_sector_statistics()` extracts characteristics (area, mean intensity, standard deviation) for each sector. Additionally, it calculates the Kct (sum of pixel values for perfusion intensity) and the percentage of each sector's contribution to total lung perfusion.

In [5]:
def compute_sector_statistics(sectors, original_image, lung_label):
    """
    For each sector, calculate area, mean intensity, and standard deviation, and derive Kct values.
    Visualize the lung with sectors and labels.
    """
    mako_colors = sns.color_palette("mako", len(sectors))
    mako_colors_rgb = [(int(c[0]*255), int(c[1]*255), int(c[2]*255)) for c in mako_colors]  # Convert to RGB

    total_intensity = 0
    sector_data = []
    combined_sectors = np.zeros((*original_image.shape, 3), dtype=np.uint8)

    for i, sector in enumerate(sectors):
        masked_pixels = original_image[sector == 255]
        area = cv2.countNonZero(sector)
        mean_intensity = masked_pixels.mean() if masked_pixels.size > 0 else 0
        std_dev_intensity = masked_pixels.std() if masked_pixels.size > 0 else 0
        total_intensity += masked_pixels.sum()  # Sum of pixel values (Kct)

        # Add sector statistics
        sector_data.append({
            'Sector': ['Upper', 'Middle', 'Lower'][i],
            'Area': area,
            'Mean Intensity': mean_intensity,
            'Standard Deviation': std_dev_intensity,
            'Kct': masked_pixels.sum()
        })

        # Combine sectors for visualization
        color = mako_colors_rgb[i]
        combined_sectors[sector > 0] = color

    # Calculate percentage contributions
    for data in sector_data:
        data['Percentage'] = (data['Kct'] / total_intensity) * 100 if total_intensity > 0 else 0

    # Plot the lung with sectors
    plt.figure(figsize=(6, 6))
    plt.imshow(combined_sectors)
    plt.title(f"{lung_label} with Sectors")
    plt.axis("off")
    plt.show()

    return sector_data, total_intensity

## Wrapper function to process the entire lung image
The results include:<br>
- Total Kct: The total perfusion intensity for each lung.<br>
- Sector Statistics: For each sector (upper, middle, lower), the area, mean intensity, standard deviation, Kct value, and percentage contribution to total perfusion.

In [6]:
def process_lung_image(filepath, projection_type):
    """
    Process the lung image and assign labels dynamically based on projection type.
    """
    # Step 1: Load and binarize image
    binary_image = load_and_binarize_image(filepath)

    # Step 2: Apply morphological operations
    processed_image = apply_morphological_operations(binary_image)

    # Step 3: Extract connected components (lungs)
    lung_masks = extract_connected_lung_components(processed_image)

    # Load original grayscale image for intensity calculations
    grayscale_image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

    # Dynamic lung labels based on projection type
    if projection_type == "anterior":
        lung_labels = ["Left Lung", "Right Lung"]
    elif projection_type == "posterior":
        lung_labels = ["Left Lung", "Right Lung"]
    else:
        raise ValueError("Invalid projection type. Use 'anterior' or 'posterior'.")

    # Process each lung
    lung_results = []
    for lung_mask, lung_label in zip(lung_masks, lung_labels):
        # Step 4: Define sectors
        sectors = define_adaptive_lung_sectors(lung_mask)

        # Step 5: Extract characteristics and plot
        stats, total_kct = compute_sector_statistics(sectors, grayscale_image, lung_label)
        lung_results.append({
            'Lung Label': lung_label,
            'Sector Statistics': stats,
            'Total Kct': total_kct
        })

    return lung_results

## Process both anterior and posterior projections

In [7]:
anterior_filepath = "ANT PERF.tif"
posterior_filepath = "POST PERF.tif"

print("Processing Anterior Lung Projection:")
anterior_results = process_lung_image(anterior_filepath, projection_type="anterior")
for lung in anterior_results:
    print(f"\n{lung['Lung Label']} - Total Kct: {lung['Total Kct']}")
    print("-" * 40)
    for sector_stat in lung['Sector Statistics']:
        print(f"Sector: {sector_stat['Sector']}")
        print(f"  - Area: {sector_stat['Area']} pixels")
        print(f"  - Mean Intensity: {sector_stat['Mean Intensity']:.2f}")
        print(f"  - Standard Deviation: {sector_stat['Standard Deviation']:.2f}")
        print(f"  - Kct (Intensity Sum): {sector_stat['Kct']}")
        print(f"  - Percentage of Total Kct: {sector_stat['Percentage']:.2f}%")
        print("-" * 40)

print("\nProcessing Posterior Lung Projection:")
posterior_results = process_lung_image(posterior_filepath, projection_type="posterior")
for lung in posterior_results:
    print(f"\n{lung['Lung Label']} - Total Kct: {lung['Total Kct']}")
    print("-" * 40)
    for sector_stat in lung['Sector Statistics']:
        print(f"Sector: {sector_stat['Sector']}")
        print(f"  - Area: {sector_stat['Area']} pixels")
        print(f"  - Mean Intensity: {sector_stat['Mean Intensity']:.2f}")
        print(f"  - Standard Deviation: {sector_stat['Standard Deviation']:.2f}")
        print(f"  - Kct (Intensity Sum): {sector_stat['Kct']}")
        print(f"  - Percentage of Total Kct: {sector_stat['Percentage']:.2f}%")
        print("-" * 40)

Processing Anterior Lung Projection:


NameError: name 'cv2' is not defined